#   

# 作業目標:

    1. 藉由固定的 dataset, 來驗證不同loss function
    2. Dataset 的特性跟我們選用的loss function 對accrancy 的影響
    
    
# 作業重點: 
    請分別選用 "MSE", "binary _crossentropy"
    查看Train/test accurancy and loss rate
    

# 導入必要的函數

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
import mxnet as mx
import mxnet.gluon as gl
import numpy as np
import sys
sys.path.append('D:/MyDeepLearning/Company/Mxnet/MxnetGluon')
import gluonbook as gb
np.random.seed(10)

# 資料準備

In [2]:
#取得Keras Dataset
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

In [3]:
#確認 CIFAR10 Dataset 資料維度
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape) 
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape) 

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)
test  data: images: (10000, 32, 32, 3)  labels: (10000, 1)


In [4]:
#資料正規化
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0

x_img_train_normalize = mx.nd.array(x_img_train_normalize).reshape((-1,3,32,32))

x_img_test_normalize = mx.nd.array(x_img_test_normalize).reshape((-1,3,32,32))




In [5]:
x_img_train_normalize.shape


(50000, 3, 32, 32)

In [23]:
#針對Label 做 ONE HOT ENCODE
#y_label_train_OneHot = tf.keras.backend.one_hot(y_label_train,10)
#y_label_test_OneHot = tf.keras.backend.one_hot(y_label_test,10)

#y_label_train_OneHot = mx.nd.array(y_label_train_OneHot)

#y_label_test_OneHot = mx.nd.array(y_label_test_OneHot)

y_label_train = mx.nd.array(y_label_train).reshape((-1,))
y_label_test = mx.nd.array(y_label_test).reshape((-1,))



# 建立模型

In [24]:
batch_size = 128
dataset = gl.data.ArrayDataset(x_img_train_normalize, y_label_train)
data_iter = gl.data.DataLoader(dataset, batch_size, shuffle=True)


In [25]:
model = gl.nn.Sequential()

In [26]:
#卷積層1

In [27]:

model.add(gl.nn.Conv2D(channels = 32, kernel_size = 3, activation = 'relu',padding = 1))


In [28]:
model.add(gl.nn.Dropout(rate=0.25))

In [29]:
#卷積層2與池化層2

In [30]:
model.add(gl.nn.MaxPool2D(pool_size=2))

In [31]:
model.add(gl.nn.Conv2D(channels = 64, kernel_size = 3, activation = 'relu', padding = 1))
model.add(gl.nn.Dropout(rate=0.25))
model.add(gl.nn.MaxPool2D(pool_size=2))

In [32]:
model.add(gl.nn.Flatten())

In [33]:
model.add(gl.nn.Dropout(rate=0.25))

In [34]:
model.add(gl.nn.Dense(1024,activation = 'relu'))
model.add(gl.nn.Dropout(rate=0.25))

In [35]:
model.add(gl.nn.Dense(10))

In [36]:
model.initialize(mx.init.Normal(sigma=0.01))

In [37]:
X = mx.nd.random.uniform(shape=(10000, 3, 32, 32))

for layer in model:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)


conv0 output shape:	 (10000, 32, 32, 32)
dropout0 output shape:	 (10000, 32, 32, 32)
pool0 output shape:	 (10000, 32, 16, 16)
conv1 output shape:	 (10000, 64, 16, 16)
dropout1 output shape:	 (10000, 64, 16, 16)
pool1 output shape:	 (10000, 64, 8, 8)
flatten0 output shape:	 (10000, 4096)
dropout2 output shape:	 (10000, 4096)
dense0 output shape:	 (10000, 1024)
dropout3 output shape:	 (10000, 1024)
dense1 output shape:	 (10000, 10)


# 載入之前訓練的模型

In [38]:
try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")

載入模型失敗!開始訓練一個新模型


# 訓練模型

In [39]:
'''
作業:
請分別選用 "MSE", "binary _crossentropy"
查看Train/test accurancy and loss rate
'''

'\n作業:\n請分別選用 "MSE", "binary _crossentropy"\n查看Train/test accurancy and loss rate\n'

In [40]:


loss = gl.loss.SoftmaxCrossEntropyLoss()

trainer = gl.Trainer(model.collect_params(), 'sgd', {'learning_rate':0.05})

num_epochs = 12
 
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with mx.autograd.record():
            l = loss(model(X), y)
        l.backward()
        trainer.step(batch_size)
        
    acc = gb.ClassifierMetrics(y_label_test,model(x_img_test_normalize))
    
    print(acc)




0.13
0.2089
0.221
0.2625
0.299
0.3384
0.3568
0.3576
0.3586
0.407
0.4247
0.4404
